In [7]:
import pandas as pd
from openpyxl import load_workbook

# Data to fill in template
data_path = 'Master data.xlsx'
data = pd.read_excel(data_path, sheet_name='Sheet1', engine='openpyxl')
df_data = pd.DataFrame(data)

In [12]:
template_path = 'adjustment template.xlsx'

# Load the supervisor list
chief_list = pd.read_excel(data_path, sheet_name='chief list', engine='openpyxl')
df_chief_list = pd.DataFrame(chief_list)

# Load the data, two parts: Name column, skip two columns, then the remaining
df_data_filtered = data.loc[0:data.shape[0], ['Name']].join(data.loc[0:data.shape[0], 'Supervisor':'Comment'])

In [17]:
from openpyxl.styles import PatternFill, Protection
from openpyxl.formatting.rule import FormulaRule
# Define the fills
grey_fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
orange_fill = PatternFill(start_color='FFC000', end_color='FFC000', fill_type='solid')
yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
mild_green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
mild_red_fill = PatternFill(start_color='FFC0CB', end_color='FFC0CB', fill_type='solid')
mild_blue_fill = PatternFill(start_color='DDEBF7', end_color='DDEBF7', fill_type='solid')


# Loop through each chief and filter the data
for index, row in df_chief_list.iterrows():
    chief = row['name']
    dept = row['department']
    wb = load_workbook(template_path)
    ws = wb['Sheet1']

    filtered_data = df_data_filtered[df_data_filtered['Supervisor'].str.contains(chief, case=False, na=False)]
    last_non_empty_row = filtered_data.shape[0] + 2 # plus two for 2 title rows
    start_row = 3 # paste data from row 3
    
    # Replace template by data
    for i, row in enumerate(filtered_data.itertuples(), start=start_row):
        for j, value in enumerate(row[1:-1], start=1):  # row[1:] to skip the index
            ws.cell(row=i, column=j, value=value)
        ws.cell(row=i, column=7, value=f"=IFERROR(F{i}/E{i},0)")
    # comments
    for i, comment in enumerate(filtered_data['Comment'], start=start_row):
        ws.cell(row=i, column=8, value=comment)  
        
    # Apply conditional formatting to ELIGIBLE
    ws.conditional_formatting.add(f'E{start_row}:E{last_non_empty_row}', FormulaRule(formula=[f'C{start_row}="No"'], fill=grey_fill))
    ws.conditional_formatting.add(f'F{start_row}:F{last_non_empty_row}', FormulaRule(formula=[f'C{start_row}="No"'], fill=grey_fill))
    ws.conditional_formatting.add(f'G{start_row}:G{last_non_empty_row}', FormulaRule(formula=[f'C{start_row}="No"'], fill=grey_fill))
    # Conditional formatting for input numbers
    ws.conditional_formatting.add(f'G{start_row}:G{last_non_empty_row}', FormulaRule(formula=[f'G{start_row}>=1.4'], fill=orange_fill))
    ws.conditional_formatting.add(f'G{start_row}:G{last_non_empty_row}', 
                                  FormulaRule(formula=[f'AND(G{start_row}>=0, G{start_row}<=0.6)'], fill=orange_fill))
    # default formatting for untouched cells
    ws.conditional_formatting.add(f'F{start_row}:F{last_non_empty_row}', FormulaRule(formula=[f'F{start_row}<=0'], fill=yellow_fill))
    
    # Banded lines
    for row in range(2, last_non_empty_row + 1, 2):
        for cell in ws[row]:
            cell.fill = mild_blue_fill
    
    # FUNDING POOL FORMAT 
    # insert formula
    ws.cell(row=1, column=6, value=f"=SUM(E3:E{last_non_empty_row})-SUM(F3:F{last_non_empty_row})")
    ws.conditional_formatting.add('F1', FormulaRule(formula=['F1<>0'], fill=mild_red_fill))
    ws.conditional_formatting.add('F1', FormulaRule(formula=['F1=0'], fill=mild_green_fill))

    # Protect and lock columns A to E and G
    for col in ws.iter_cols(min_col=1, max_col=5):
        for cell in col:
            cell.protection = Protection(locked=True)
    for cell in ws['G']:
        cell.protection = Protection(locked=True)

    # Ensure columns F and H are unlocked
    for cell in ws['F']:
        cell.protection = Protection(locked=False)
    for cell in ws['H']:
        cell.protection = Protection(locked=False)

    # Protect the first two rows
    for row in ws.iter_rows(min_row=1, max_row=2):
        for cell in row:
            cell.protection = Protection(locked=True)
    
    # Lock column F and H if eligible is "No"
    for i in range(start_row, last_non_empty_row + 1):
        if ws.cell(row=i, column=3).value == "No ":
            ws.cell(row=i, column=6).protection = Protection(locked=True) 
            ws.cell(row=i, column=8).protection = Protection(locked=True) 
    ws.protection.sheet = True

    output_path = r'output//'+f"{dept}_INCENTIVES_2024.xlsx"
    wb.save(output_path)

In [ ]:
# HTML body
html_body = f"""
<!DOCTYPE html>
<html>
  <body style="font-family: Arial, sans-serif; font-size: 14px; color: #000;">
    <p>Dear Name:</p>

    <p>Please see attachment.</p>

    <p>Please contact 
      <a href="mailto:aaa@example.com" style="color: #1a0dab; text-decoration: none;">mailto:aaa@example.com</a> for any question. 
    </p>

    <ol>
      <li>Visit <a href="https://alumni.jhu.edu" style="color: #1a0dab;">alumni.jhu.edu</a> for instruction.</li>
      <li>Fill the spreadsheet.</li>
    </ol>
    
    
    <p>Best, </p>
    <p>me</p>
  </body>
</html>

"""


In [103]:
import win32com.client as win32
excel_folder=r'output//'
emails_folder=r'output//'
outlook=win32.Dispatch('Outlook.Application')

import datetime

def email(dept,chiefemail,firstname,chiefname):
#     mail=outlook.CreateItemFromTemplate(email_template) - use this is and .oft template available
    mail.Subject='[Confidential] '+dept+' Incentives Worksheet - Deadline 3/31'
    mail.HTMLBody = html_body
    mail.To=chiefemail
    mail.cc='xxx@gmail.com; yyy@gmail.com'
    mail.Importance = 2
    mail.TaskDueDate = datetime.datetime(2024, 3, 31, 12, 0)  # Set due date to March 31, 2024 at 12:00 PM
    mail.HTMLBody=mail.HTMLBody.replace('Name',firstname)
    mail.Attachments.Add(Source=excel_folder+dept.rstrip()+"_INCENTIVES_2024.xlsx")
    # mail.Display(True)
    mail.SaveAs(emails_folder+chiefname.rstrip()+".oft")

for i in range(len(df_chief_list)):
    try:
        email(df_chief_list["department"][i],df_chief_list["email"][i],df_chief_list["first name"][i],df_chief_list["SUPERVISOR NAME"][i])
    except Exception as e:
        print(f"An error occurred: {e}",df_chief_list["SUPERVISOR NAME"][i])
        
print('Completed.')

Completed.
